# Investigating Fandango Movie Ratings
In October 2015, a data journalist named Walt Hickey analyzed Fandango's aggregated movie ratings data and found strong evidence to suggest that Fandango's rating system was biased and dishonest. He published his analysis in [this article](https://fivethirtyeight.com/features/fandango-movies-ratings/) — a great piece of data journalism worth reading.


In this project, we'll analyze more recent movie ratings data to determine whether there has been any change in Fandango's rating system since Hickey's analysis.

## Data
One of the best ways to figure out whether there has been any change in Fandango's rating system after Hickey's analysis is to compare the system's characteristics previous and after the analysis. Fortunately, we have ready-made data for both these periods of time:

Walt Hickey made the data he analyzed publicly available on GitHub which can be found [here](https://github.com/fivethirtyeight/data/tree/master/fandango).

- **FILM**	The film in question
- **RottenTomatoes**	The Rotten Tomatoes Tomatometer score for the film
- **RottenTomatoes_User**	The Rotten Tomatoes user score for the film
- **Metacritic**	The Metacritic critic score for the film
- **Metacritic_User**	The Metacritic user score for the film
- **IMDB**	The IMDb user score for the film
- **Fandango_Stars**	The number of stars the film had on its Fandango movie page
- **Fandango_Ratingvalue**	The Fandango ratingValue for the film, as pulled from the HTML of each page. This is the actual average score the movie obtained.
- **RT_norm**	The Rotten Tomatoes Tomatometer score for the film , normalized to a 0 to 5 point system
- **RT_user_norm**	The Rotten Tomatoes user score for the film , normalized to a 0 to 5 point system
- **Metacritic_norm**	The Metacritic critic score for the film, normalized to a 0 to 5 point system
- **Metacritic_user_nom**	The Metacritic user score for the film, normalized to a 0 to 5 point system
- **IMDB_norm**	The IMDb user score for the film, normalized to a 0 to 5 point system
- **RT_norm_round**	The Rotten Tomatoes Tomatometer score for the film , normalized to a 0 to 5 point system and rounded to the nearest half-star
- **RT_user_norm_round**	The Rotten Tomatoes user score for the film , normalized to a 0 to 5 point system and rounded to the nearest half-star
- **Metacritic_norm_round**	The Metacritic critic score for the film, normalized to a 0 to 5 point system and rounded to the nearest half-star
- **Metacritic_user_norm_round**	The Metacritic user score for the film, normalized to a 0 to 5 point system and rounded to the nearest half-star
- **IMDB_norm_round**	The IMDb user score for the film, normalized to a 0 to 5 point system and rounded to the nearest half-star
- **Metacritic_user_vote_count**	The number of user votes the film had on Metacritic
- **IMDB_user_vote_count**	The number of user votes the film had on IMDb
- **Fandango_votes**	The number of user votes the film had on Fandango
- **Fandango_Difference**	The difference between the presented Fandango_Stars and the actual Fandango_Ratingvalue


Dataquest collected movie ratings data for movies released in 2016 and 2017, after Hickey's analysis, and published it on Github [here](https://github.com/mircealex/Movie_ratings_2016_17).

- **movie**	the name of the movie
- **year**	the release year of the movie
- **metascore**	the Metacritic rating of the movie (the "metascore" - critic score)
- **imdb**	the IMDB rating of the movie (user score)
- **tmeter**	the Rotten Tomatoes rating of the movie (the "tomatometer" - critic score)
- **audience**	the Rotten Tomatoes rating of the movie (user score)
- **fandango	the Fandango rating of the movie (user score)
- **n_metascore	the metascore normalized to a 0-5 scale
- **n_imdb	the IMDB rating normalized to a 0-5 scale
- **n_tmeter	the tomatometer normalized to a 0-5 scale
- **n_audience	the Rotten Tomatoes user score normalized to a 0-5 scale
- **nr_metascore	the metascore normalized to a 0-5 scale and rounded to the nearest 0.5
- **nr_imdb	the IMDB rating normalized to a 0-5 scale and rounded to the nearest 0.5
- **nr_tmeter	the tomatometer normalized to a 0-5 scale and rounded to the nearest 0.5
- **nr_audience	the Rotten Tomatoes user score normalized to a 0-5 scale and rounded to the nearest 0.5

We'll use both of these datasets to analyze the rating system's characteristics before and after Hickey's analysis.